In [1]:
import warnings
# warnings.filterwarnings("ignore", message="numpy.dtype size changed")
# warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import KFold
# from MTS import *
# from orthogonal_array import *
from sklearn.mixture import BayesianGaussianMixture

In [2]:
def bgmm(X_train, X_test):
    bgmm = BayesianGaussianMixture(n_components=len(X_train.columns), verbose=0, max_iter=1000, random_state=0)
    bgmm.fit(X_train)
    score = bgmm.score_samples(X_test)
    return score

def break_even(a, y_test):
    """ 分岐点精度の計算 """
    # 正常標本精度と異常標本精度の計算
    y_test.reset_index(drop=True, inplace=True) # インデックスリセット
    idx = a.argsort()[::-1] # 降順のインデックス計算
    n_total = len(y_test)
    n_anom = sum(y_test)
    n_norm = n_total - n_anom
    coverage = np.zeros(n_total) # 異常標本精度
    detection = np.zeros(n_total) # 正常標本精度
    for i in range(n_total):
        n_detected_anom = sum(y_test[idx][:i])
        n_detected_norm = n_total - i - sum(y_test[idx][i:])
        coverage[i] = n_detected_anom / n_anom
        detection[i] = n_detected_norm / n_norm

    # 分岐点精度の計算
    thresh = 0
    for i, (c_score, d_score) in enumerate(zip(coverage, detection)):
        if c_score >= d_score:
            thresh = i
            break
    break_even_point = a[idx][thresh]
    print(break_even_point, c_score, d_score)
    return (break_even_point, c_score), (coverage, detection)

In [3]:
# ファイル読み込み
IN_FILE = '../data/SkillCraft1_Dataset.csv'
df = pd.read_csv(IN_FILE, header=0, index_col=None, sep=',')

In [4]:
# '?'を含む行を削除
df = df[(df=='?').sum(axis=1)==0]

In [5]:
# シャッフルとインデックス更新
df = df.sample(frac=1, random_state=0) # 行シャッフル
df = df.reset_index(drop=True) # インデックスの更新

In [6]:
# データセットの作成
target_col = 'LeagueIndex'
target = 1
del_col = [target_col, 'GameID']
X = df.drop(del_col, axis=1)
y = (df[target_col]==target).astype(np.int32)
print(X.shape)

(3338, 18)


# 異常検出（5分割交差検証による評価）

In [7]:
%%time
# 項目選択のないMT法（単なるMahalanobis距離）交差確認
scores = []
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # 混合正規分布による異常検知
    # →平均0.3957
    anomaly_score = -bgmm(X_train[y_train==0], X_test)
    (break_even_point, score), (coverage, detection) = break_even(anomaly_score, y_test)
    anomaly_score_abnormal = anomaly_score[y_test==1]
    scores.append(score)
print('average: {}'.format(sum(scores)/len(scores)))

-11.356159793619181 0.5666666666666667 0.5658307210031348
-11.67012703500609 0.5714285714285714 0.5576619273301737
-10.257279954271286 0.6875 0.6871069182389937
-11.22550127955575 0.6060606060606061 0.5788643533123028
-10.388488472305554 0.6756756756756757 0.6714285714285714
average: 0.6214663039663039
Wall time: 25.5 s
